In [67]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
import pandas as pd

In [68]:
df = pd.read_csv("COLL_TEC_CONSOLIDADO.txt", delimiter=",", encoding="latin-1", low_memory=False)
df_copy = df.copy()

In [69]:
X = df_copy.drop(columns='Variable_objetivo')
y = df_copy['Variable_objetivo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [70]:
print(len(X_train))
print(len(y_test))

1031904
257977


In [71]:
def limpieza_sin_categoricas(df_copy):
    df_copy = df_copy.copy()

    ## Convertir columnas que empiezan con "Fecha" o "Prox" a tipo fecha
    for column in df_copy.columns:
        if column.startswith('Fecha') or column.startswith('Prox'):
            df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')

    # Agregar las 6 columnas con valor inicial en 1
    for i in range(1, 7):
        df_copy[f'Activo_M{i}'] = 1

    meses = range(1, 7)
    for j in meses:
        columnas_check = [f'Saldo_total_M{k}' for k in range(j, 7)]
        columnas_fill = [
            f'Ciclo_atraso_M{j}', f'Pago_M{j}', f'Fecha_pago_M{j}', f'Utilizacion_M{j}',
            f'Fecha_corte_M{j}', f'Fecha_limite_pago_M{j}'
        ]
        mask_nulos = df_copy[columnas_check].isnull().all(axis=1)
        df_copy.loc[mask_nulos, columnas_fill] = df_copy.loc[mask_nulos, columnas_fill].fillna(0)
        df_copy.loc[mask_nulos, f'Activo_M{j}'] = 0

    if 'Fecha_prox_corte_M1' in df_copy.columns:
        mask_fecha_prox = df_copy[[f'Saldo_total_M{k}' for k in meses]].isnull().all(axis=1)
        df_copy.loc[mask_fecha_prox, 'Fecha_prox_corte_M1'] = 0

    # Drop columnas 'Behavior'
    df_copy = df_copy.drop(df_copy.filter(regex='Behavior').columns, axis=1)

    # Canal_Pago y Canal_Pago_M1-M6
    columnas_canal_pago = [f'Canal_Pago_M{j}' for j in meses]
    df_copy[columnas_canal_pago] = df_copy[columnas_canal_pago].fillna('Desconocido')
    df_copy['Moda_Canal_Pago'] = df_copy[columnas_canal_pago].mode(axis=1)[0]
    df_copy['Canal_Pago'].fillna(df_copy['Moda_Canal_Pago'], inplace=True)
    df_copy.drop(columns=['Moda_Canal_Pago'], inplace=True)

    # Eliminar columnas de Fecha_prox_corte_M2-M6 si coinciden con fechas corte previas
    mask_fecha_igual = False
    for i in range(2, 7):
        if f'Fecha_prox_corte_M{i}' in df_copy.columns and f'Fecha_corte_M{i-1}' in df_copy.columns:
            mask_fecha_igual |= (df_copy[f'Fecha_prox_corte_M{i}'] == df_copy[f'Fecha_corte_M{i-1}'])

    df_copy = df_copy[mask_fecha_igual]
    columnas_a_eliminar = [f'Fecha_prox_corte_M{i}' for i in range(2, 7)]
    df_copy.drop(columns=[col for col in columnas_a_eliminar if col in df_copy.columns], inplace=True)

    # Convertir fechas a días desde fecha base
    fecha_base = pd.to_datetime('01/01/01')
    for i in range(1, 7):
        for col in [f'Fecha_corte_M{i}', f'Fecha_limite_pago_M{i}', f'Fecha_pago_M{i}']:
            if col in df_copy.columns:
                mask_zeros = df_copy[col] == 0
                df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')
                df_copy[col] = (df_copy[col] - fecha_base).dt.days
                df_copy.loc[mask_zeros, col] = 0

    # Saldo_total_M1, Saldo_Mes_M1, Pago_minimo_M1
    mask_nulos_m1 = df_copy[['Saldo_total_M1', 'Saldo_Mes_M1', 'Pago_minimo_M1']].isnull().all(axis=1)
    columnas_m1 = df_copy.columns[df_copy.columns.str.contains('M1')]
    df_copy.loc[mask_nulos_m1, columnas_m1] = df_copy.loc[mask_nulos_m1, columnas_m1].fillna(0)

    # Eliminar columnas innecesarias
    df_copy.drop(df_copy.filter(regex='Genero').columns, axis=1, inplace=True)
    df_copy.drop(df_copy.filter(regex='Fecha_pago').columns, axis=1, inplace=True)
    for i in range(1, 7):
        df_copy.drop(df_copy.filter(regex=f'Pago_M{i}').columns, axis=1, inplace=True)
    df_copy.drop(columns=['Pago'], errors='ignore', inplace=True)

    # Crear columnas Deuda_M1 a Deuda_M6
    for i in range(1, 7):
        df_copy[f'Deuda_M{i}'] = df_copy['Limite_credito'] * df_copy[f'Utilizacion_M{i}']

    # Eliminar filas con datos faltantes en columnas críticas
    columnas_a_verificar = sum([[f'Saldo_total_M{i}', f'Saldo_Mes_M{i}', f'Pago_minimo_M{i}'] for i in range(1, 7)], [])
    df_copy = df_copy.dropna(subset=columnas_a_verificar)
    df_copy.dropna(inplace=True)

    # Convertir columnas tipo fecha restantes a días desde fecha_base
    for column in df_copy.columns:
        if column.startswith('Fecha') and not column.endswith(('M1', 'M2', 'M3', 'M4', 'M5', 'M6')):
            df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
        if column.startswith('Prox'):
            df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')

    df_dates = df_copy.select_dtypes(include=['datetime64[ns]'])
    for col in df_dates.columns:
        mask_zeros = df_copy[col] == 0
        df_copy[col] = (df_copy[col] - fecha_base).dt.days
        df_copy.loc[mask_zeros, col] = 0

    # Especial: Fecha_prox_corte_M1
    if 'Fecha_prox_corte_M1' in df_copy.columns:
        mask_zeros_2 = df_copy['Fecha_prox_corte_M1'] == 0
        df_copy['Fecha_prox_corte_M1'] = pd.to_datetime(df_copy['Fecha_prox_corte_M1'], errors='coerce')
        df_copy['Fecha_prox_corte_M1'] = (df_copy['Fecha_prox_corte_M1'] - fecha_base).dt.days
        df_copy['Fecha_prox_corte_M1'] = df_copy['Fecha_prox_corte_M1'].fillna(0).astype(int)

    # No se codifican las variables categóricas aquí (eso se hará luego en el pipeline)

    return df_copy


In [72]:
limpieza_transformer = FunctionTransformer(limpieza_sin_categoricas)

class Preprocesador(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ohe = None
        self.scaler = None
        self.categorical_cols = None
        self.numeric_cols = None

    def fit(self, X, y=None):
        self.categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        self.numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

        self.ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
        self.ohe.fit(X[self.categorical_cols])

        self.scaler = StandardScaler()
        self.scaler.fit(X[self.numeric_cols])

        return self

    def transform(self, X):
        X = X.copy()

        X_cat = pd.DataFrame(self.ohe.transform(X[self.categorical_cols]),
                             columns=self.ohe.get_feature_names_out(self.categorical_cols),
                             index=X.index)

        X_num = pd.DataFrame(self.scaler.transform(X[self.numeric_cols]),
                             columns=self.numeric_cols,
                             index=X.index)

        return pd.concat([X_num, X_cat], axis=1)


In [73]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class ToNamedDataFrame(BaseEstimator, TransformerMixin):
    def __init__(self, prefix="PCA_", n_components=70):
        self.prefix = prefix
        self.n_components = n_components

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        column_names = [f"{self.prefix}{i+1}" for i in range(self.n_components)]
        return pd.DataFrame(X, columns=column_names)

In [74]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
import pandas as pd

class PCAWithTarget(BaseEstimator, TransformerMixin):
    def __init__(self, target_column='Variable_objetivo', n_components=70):
        self.target_column = target_column
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components)

    def fit(self, X, y=None):
        X_features = X.drop(columns=[self.target_column])
        self.pca.fit(X_features)
        return self

    def transform(self, X):
        X_features = X.drop(columns=[self.target_column])
        y_target = X[self.target_column].reset_index(drop=True)

        X_pca = self.pca.transform(X_features)
        df_pca = pd.DataFrame(X_pca, columns=[f'PCA_{i+1}' for i in range(self.n_components)])
        df_pca[self.target_column] = y_target.values

        return df_pca


In [75]:
pipeline1 = Pipeline([
    ("Limpieza", limpieza_transformer),
    ("Preprocesamiento", Preprocesador()),
    ('pca', PCA(n_components=70)),
    ("ToName", ToNamedDataFrame())
])

In [76]:
pipeline = Pipeline([
    ("Limpieza", limpieza_transformer),
    ("Preprocesamiento", Preprocesador()),
    ("PCAConY", PCAWithTarget(target_column='Variable_objetivo', n_components=70))
])

In [77]:
df_train = pd.concat([X_train, y_train], axis=1)
pipeline.fit(df_train)
X_train_transformed = pipeline.transform(df_train)
df_result = pd.DataFrame(X_train_transformed)
print(df_result.head())

C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format 

      PCA_1     PCA_2     PCA_3     PCA_4     PCA_5     PCA_6     PCA_7  \
0 -0.271324  5.162117 -0.050120 -1.605712 -0.532088 -0.623101 -0.951396   
1  2.801306  4.459874  0.118025  4.540536  7.765561  0.800761 -5.109839   
2 -2.071284 -0.126923 -0.045793 -1.702465  0.216788 -0.039008 -0.902470   
3 -2.430398 -2.150539 -0.033882  1.185870 -1.006279 -0.289071  0.624520   
4 -3.165731 -5.401905 -0.048627 -1.536754 -0.587528 -0.240524 -0.782035   

      PCA_8     PCA_9    PCA_10  ...    PCA_62    PCA_63    PCA_64    PCA_65  \
0  0.002424 -0.177764 -1.618186  ...  0.020030 -0.050897  0.011333  0.032061   
1  6.573827 -1.284922 -1.775453  ...  0.013140  0.033034 -0.011095 -0.046039   
2  1.422530  0.918510 -0.569859  ...  0.014079  0.025667 -0.006790 -0.013520   
3 -0.104528 -0.831105 -0.061387  ... -0.166113  0.043254 -0.005385 -0.073138   
4  0.332962  1.360690 -0.416500  ... -0.033806  0.070218  0.003640  0.059123   

     PCA_66    PCA_67    PCA_68    PCA_69    PCA_70  Variable_objeti

c:\Users\admon\Documents\Python2.0\venv\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [79]:
df_test = pd.concat([X_test, y_test], axis=1)
X_test_transformed = pipeline.transform(df_test)

C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy[column] = pd.to_datetime(df_copy[column], errors='coerce')
C:\Users\admon\AppData\Local\Temp\ipykernel_18652\359942081.py:7: UserWarning: Parsing dates in %d/%m/%Y format 

In [ ]:
import joblib
model_imported = joblib.load('random_forest_model (1).pkl')
prediction = model_imported.predict(X_test_transformed)


c:\Users\admon\Documents\Python2.0\venv\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\admon\Documents\Python2.0\venv\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [83]:
df_xtes = pd.DataFrame(X_test_transformed)
prediction = model_imported.predict(np.array(df_xtes.iloc[0]).reshape(1,-1))
prediction

c:\Users\admon\Documents\Python2.0\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([2])

In [ ]:
df_xtes.head()

,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,...,PCA_62,PCA_63,PCA_64,PCA_65,PCA_66,PCA_67,PCA_68,PCA_69,PCA_70,Variable_objetivo
0,-2.297008,-1.727632,0.044886,2.041908,0.397214,1.408909,2.591787,0.798531,-1.193392,0.647039,...,0.041650,-0.098924,-0.014700,-0.036470,0.001011,-0.111173,0.008986,0.004182,0.065582,-0.686946
1,1.706153,4.783324,-0.047227,-0.564551,-1.156659,0.153699,1.132081,0.454979,1.506958,0.182231,...,-0.090484,-0.107445,0.014660,-0.033938,-0.132079,-0.009257,-0.013440,-0.002312,-0.119636,-0.686946
2,8.484697,-1.854434,-0.154647,-5.076267,-2.032206,-0.908501,-1.690332,-0.503924,0.075719,0.772982,...,-0.007191,0.004409,-0.040649,-0.022979,0.057517,-0.037666,-0.026373,0.002796,-0.011346,1.455719
3,3.437193,1.446923,0.025115,1.884805,-0.160923,1.169257,2.728692,-0.443787,1.385379,0.426723,...,0.264779,-0.301061,0.009189,-0.010021,-0.207674,-0.183845,0.010838,-0.011214,0.031518,-0.686946
4,6.764417,3.685242,-0.117548,-0.180082,-1.972628,-0.665896,0.282524,-0.798051,-0.056902,-0.490922,...,-0.004047,0.016839,-0.002769,-0.014589,0.041828,-0.021093,-0.009631,0.006829,-0.248378,-0.686946
